<a href="https://colab.research.google.com/github/hayleypc/HawaiiClimate/blob/main/soil_dl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import geopandas as gpd

Data preprocessing steps below to prepare for model training:


1.   only using the 0-20 measured or imputed SOC values
2.   minmax scalar normalizes all data between 0 and 1


The build_model function defines a neural network using Keras' Sequential model, so that the layers of the model are arranged in a sequence. Here's a breakdown of each component of the model:

1.  Input Shape and First Dense Layer:

*   Dense(256, activation='relu', input_dim=input_shape)

This is the first layer of the model, a fully connected (Dense) layer with 256 neurons. It uses the ReLU (Rectified Linear Unit) activation function. The input_dim parameter is set to input_shape, which specifies the shape of the input data the network expects. This layer takes the input data and begins the process of extracting features by applying weights, biases, and the ReLU activation.

2.   Batch Normalization:

This layer normalizes the activations of the previous layer at each batch, i.e., it applies a transformation that maintains the mean output close to 0 and the output standard deviation close to 1. This helps in stabilizing the learning process and reducing the number of training epochs required to train deep networks.


3. Leaky ReLU Activation:  

*  LeakyReLU(alpha=0.2)

This is an activation layer that allows a small gradient when the unit is not active and the output is less than zero, which helps to keep the information flowing through the network. alpha is a coefficient for negative values, allowing a small negative slope (0.2 in this case).


4.  Dropout:

*   Dropout(0.3)

This layer randomly sets a fraction (30% here) of the input units to 0 at each update during training time, which helps to prevent overfitting.


5.  Additional Dense Layers:

Subsequent blocks of Dense, BatchNormalization, LeakyReLU, and Dropout layers follow the first block. These layers increase in size from 512 to 1024 neurons, continuing the pattern of extracting features and applying non-linearity and regularization techniques. Each dense layer serves to further refine the features extracted by the previous layers, with larger layers potentially capturing more complex patterns in the data.


6.   Output Layer:


*  Dense(1, activation='linear'):
This is the output layer of the model. It has 1 neuron, and it uses a linear activation function, which means the output is a direct weighted sum of the inputs. This type of output is typical for regression tasks or any task where you need a single continuous output value from the model.

Overall, this model architecture is typical for a deep learning task that requires capturing complex relationships in the data through successive layers of transformation and non-linear activation, combined with techniques to prevent overfitting (Batch Normalization and Dropout).


Why 256 neurons? The choice of 256 neurons in the first layer of a neural network is a bit arbitrary but can be grounded in several practical and theoretical considerations:

Power of Two: Often, the number of neurons in layers of neural networks are chosen to be powers of two (e.g., 64, 128, 256, 512). This choice is largely driven by computational efficiency—many hardware architectures and underlying mathematical libraries are optimized to handle data sizes that are powers of two more efficiently.
Model Capacity: The number of neurons in a layer determines its capacity to capture information. With 256 neurons, the layer can potentially learn more complex patterns in the data compared to a layer with fewer neurons. However, increasing the number of neurons also increases the risk of overfitting, especially if the amount of training data is not sufficiently large. Balancing this with techniques such as dropout and regularization is crucial.
Trade-off Between Performance and Complexity: Choosing the size of a layer involves balancing the computational cost and the model's performance. A larger layer (more neurons) can model more complex functions but requires more computation both in terms of memory and processing power. Conversely, a smaller layer may be less capable but more efficient. The number 256 is often seen as a good compromise in many applications, offering substantial modeling power without excessive computational demands.
Empirical Experience: Often, the choice of architecture parameters such as the number of neurons in a layer comes from empirical experience and experimentation. Researchers and practitioners might start with a configuration known to perform well on similar types of problems or datasets and adjust from there based on specific task performance and computational resources.
Scalability and Flexibility: Starting with 256 neurons in the first layer provides flexibility for scaling the model up or down. Depending on the outcomes of initial experiments, it's relatively straightforward to adjust the network size by adding more neurons or layers (scaling up) or reducing them (scaling down) to better fit the task without starting from scratch.
In practice, the specific choice of 256 neurons would ideally be validated through experimentation—testing the model's performance with different configurations and tuning based on validation results to find the best balance for the specific dataset and task at hand.

the use of mean squared error as the loss function = DL model for regression

Data Preparation:
**X = scaled_numeric_df.iloc[:, :-1]:** This line selects all rows and all columns except the last column of the dataframe scaled_numeric_df and assigns it to X. X contains the independent variables (features) used to predict the target variable.
**y = scaled_numeric_df.iloc[:, -1]:** This line selects all rows of the last column of scaled_numeric_df, which is assumed to be the dependent variable (target).
Train-Test Split:
**X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42):** This function splits the dataset into training and testing sets. test_size=0.2 indicates that 20% of the data will be set aside as the test set, and the remaining 80% will be used for training. The random_state parameter ensures that the split is reproducible.
Model Building and Compilation:
**model = build_model(X_train.shape[1]):** This calls the build_model function, which constructs a neural network model. The number of input dimensions (input_shape) is set to the number of features in X_train.
model.compile(...): This configures the model for training by setting the optimizer (Adam with a specific learning rate) and the loss function (mean_squared_error, suitable for regression tasks).
Model Training:
The model is trained three times with different learning rates and for different numbers of epochs. This approach, often called learning rate annealing or step decay, gradually decreases the learning rate:
First, the model is trained for 100 epochs with a learning rate of 0.001.
Then, it is recompiled with a lower learning rate of 0.0001 and trained for an additional 300 epochs.
Finally, it is recompiled again with an even lower learning rate of 0.00001 and trained for 600 more epochs.
validation_split=0.2 reserves 20% of the training data for validation to monitor and prevent overfitting.
batch_size=128 determines the number of samples per gradient update.
verbose=1 enables verbose output in the console, showing progress logs during training.
Model Evaluation:
**test_loss = model.evaluate(X_test, y_test):** After training, the model's performance is evaluated on the unseen test dataset. This function returns the loss (mean squared error in this case) calculated on the test set.

Generate Predictions:
**predictions = model.predict(X_test).flatten():** This line uses the trained model to make predictions on the test dataset (X_test). The .flatten() method is used to convert the output array into a 1D array if necessary, ensuring that predictions is a flat array.
Reverse Scaling of Predictions:
**dummy_array = np.full((len(predictions), 28), fill_value=0.5):** Creates a dummy array with the same number of rows as predictions and 28 columns, filling it with 0.5. The number 28 and the fill value 0.5 are placeholders and should be adjusted according to specific scaling details of the original data.
**dummy_array[:, -1] = predictions:** Places the predictions into the last column of dummy_array.
**inversed_predictions = scaler.inverse_transform(dummy_array)[:, -1]:** Uses the scaler to reverse the scaling transformation applied to the numeric data, extracting only the last column which contains the scaled predictions returned to their original scale.
Reverse Scaling of True Values:
**dummy_array[:, -1] = y_test:** Replaces the last column of dummy_array with the actual test target values (y_test).
**inversed_truth = scaler.inverse_transform(dummy_array)[:, -1]:** Similarly, this line reverses the scaling of the true target values, allowing for direct comparison with the original target values in the dataset.
Compute R-Squared:
**r_squared = r2_score(inversed_truth, inversed_predictions):** Calculates the coefficient of determination
Return Statement:
**return model, test_loss, r_squared, inversed_predictions, scaler:** The function returns several objects:
model: The trained model object.
test_loss: The loss calculated on the test dataset, obtained from earlier model evaluation.
r_squared: The R-squared statistic for the predictions versus the actual test data.
inversed_predictions: The predictions of the model converted back to the original scale of the data.
scaler: The scaler object used for normalizing and reversing the normalization of the data.

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, LeakyReLU
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import r2_score

def preprocess_and_train_model(matched_data):
    # Data Preprocessing Steps
    # matched_data = matches_within_distance.groupby('unique_id').sample(n=1)
    # matched_data = matched_data.reset_index(drop=True)
    matched_data['distance'] = 0
    unique_rows = matched_data[matched_data["depth_adj_bottom"] == '20']
    unique_rows = unique_rows.drop_duplicates(subset=['latitude', 'longitude'])
    unique_rows = unique_rows[unique_rows['distance'] < 251]
    matched_data = unique_rows
    matched_data['imp_c_float'] = [float(datum) for datum in matched_data['imp_c']]
    id_fields = matched_data[['source_dataset', 'island', 'soil_column_id', 'unique_id', 'depth_top', 'depth_bottom', 'depth_adj_bottom', 'latitude', 'longitude']]
    keep_cols = [21,22,23,24,25,26,27,28, 29, 31,32, 33,34,36,36,37,38,39,40,41,42,43,44,45,46,47, 48, 55]
    numeric_cols = matched_data.iloc[:,keep_cols]
    numeric_cols.replace('', np.nan, inplace=True)
    numeric_cols = numeric_cols.astype(float)
    numeric_cols.fillna(0, inplace=True)
    scaler = MinMaxScaler()
    scaler.fit(numeric_cols)
    scaled_numeric_cols = scaler.transform(numeric_cols)
    scaled_numeric_df = pd.DataFrame(scaled_numeric_cols, columns=numeric_cols.columns, index=numeric_cols.index)
    numeric_df = pd.concat([id_fields, scaled_numeric_df], axis=1)

    # Model Definition
    def build_model(input_shape):
        model = Sequential([
            Dense(256, activation='relu', input_dim=input_shape),
            BatchNormalization(),
            LeakyReLU(alpha=0.2),
            Dropout(0.3),
            Dense(512, activation='relu'),
            BatchNormalization(),
            LeakyReLU(alpha=0.2),
            Dropout(0.3),
            Dense(1024, activation='relu'),
            BatchNormalization(),
            LeakyReLU(alpha=0.2),
            Dropout(0.3),
            Dense(1, activation='linear')  # Adjust output layer according to your needs
        ])
        return model

    X = scaled_numeric_df.iloc[:, :-1]
    y = scaled_numeric_df.iloc[:, -1]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = build_model(X_train.shape[1])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    model.fit(X_train, y_train, validation_split=0.2, epochs=100, batch_size=128, verbose=1)
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='mean_squared_error')
    model.fit(X_train, y_train, validation_split=0.2, epochs=300, batch_size=128, verbose=1)
    model.compile(optimizer=Adam(learning_rate=0.00001), loss='mean_squared_error')
    model.fit(X_train, y_train, validation_split=0.2, epochs=600, batch_size=128, verbose=1)

    test_loss = model.evaluate(X_test, y_test)

    # Predictions and Evaluation
    predictions = model.predict(X_test).flatten()
    dummy_array = np.full((len(predictions), 28), fill_value=0.5)  # Example for reverse scaling
    dummy_array[:, -1] = predictions
    inversed_predictions = scaler.inverse_transform(dummy_array)[:, -1]
    dummy_array[:, -1] = y_test
    inversed_truth = scaler.inverse_transform(dummy_array)[:, -1]
    r_squared = r2_score(inversed_truth, inversed_predictions)

    return model, test_loss, r_squared, inversed_predictions, scaler

# Use the function with your DataFrame
# returns: model, test_loss, r_squared, predictions = preprocess_and_train_model(your_dataframe)


In [5]:
def predict_on_reserve(matched_data,model,scaler):
    # Data Preprocessing Steps
    # matched_data = matches_within_distance.groupby('unique_id').sample(n=1)
    # matched_data = matched_data.reset_index(drop=True)
    matched_data['distance'] = 0
    unique_rows = matched_data[matched_data["depth_adj_bottom"] == '20']
    unique_rows = unique_rows.drop_duplicates(subset=['latitude', 'longitude'])
    unique_rows = unique_rows[unique_rows['distance'] < 251]
    matched_data = unique_rows
    matched_data['imp_c_float'] = [float(datum) for datum in matched_data['imp_c']]
    id_fields = matched_data[['source_dataset', 'island', 'soil_column_id', 'unique_id', 'depth_top', 'depth_bottom', 'depth_adj_bottom', 'latitude', 'longitude']]
    keep_cols = [21,22,23,24,25,26,27,28, 29, 31,32, 33,34,36,36,37,38,39,40,41,42,43,44,45,46,47, 48, 55]
    numeric_cols = matched_data.iloc[:,keep_cols]
    numeric_cols.replace('', np.nan, inplace=True)
    numeric_cols = numeric_cols.astype(float)
    numeric_cols.fillna(0, inplace=True)
    scaled_numeric_cols = scaler.transform(numeric_cols)
    scaled_numeric_df = pd.DataFrame(scaled_numeric_cols, columns=numeric_cols.columns, index=numeric_cols.index)
    numeric_df = pd.concat([id_fields, scaled_numeric_df], axis=1)


    X = scaled_numeric_df.iloc[:, :-1]
    y = scaled_numeric_df.iloc[:, -1]

    # Predictions and Evaluation
    predictions = model.predict(X).flatten()
    dummy_array = np.full((len(predictions), 28), fill_value=0.5)  # Example for reverse scaling
    dummy_array[:, -1] = predictions
    inversed_predictions = scaler.inverse_transform(dummy_array)[:, -1]
    dummy_array[:, -1] = y
    inversed_truth = scaler.inverse_transform(dummy_array)[:, -1]
    matched_data['inversed_predictions'] = inversed_predictions.flatten()
    matched_data['inversed_imp_c'] =inversed_truth.flatten()
    return matched_data

In [6]:
file_path = '/content/drive/My Drive/sequence_data.csv'

In [7]:
drivers_gpd = gpd.read_file('/content/drive/MyDrive/hawaii_soils/Analysis Data/250_summary_grid_dt.gpkg')

In [8]:
soils_csv = gpd.read_file('/content/drive/MyDrive/hawaii_soils/HI soils data/combined_soc_2024_04_05.csv')

In [9]:
from shapely.geometry import Point

soils_csv = soils_csv[(soils_csv['latitude'] != '') & (soils_csv['longitude'] != '')]
soils_csv['geometry'] = soils_csv.apply(lambda row: Point(float(row['longitude']), float(row['latitude'] )), axis=1)
soils_gpd = gpd.GeoDataFrame(soils_csv, geometry='geometry', crs="EPSG:4326")

In [10]:


# Ensure both GeoDataFrames have the same CRS
soils_gpd = soils_gpd.to_crs(drivers_gpd.crs)

# Perform spatial join
matched_data = gpd.sjoin_nearest(soils_gpd, drivers_gpd, how='left', distance_col='distance')

In [11]:
# Apply a buffer to each geometry in one of the GeoDataFrames (e.g., soils_gpd)
soils_buffered = soils_gpd.copy()

soils_buffered.geometry = soils_buffered.to_crs(epsg=32604).geometry.buffer(1000).to_crs('ESRI:102261')

# soils_gpd = soils_gpd
# Step 2: Spatial Join
# Perform a spatial join with the buffered GeoDataFrame
# This finds all drivers_gpd points that fall within the 10,000-meter buffer of any point in soils_gpd
matches_within_distance = gpd.sjoin(soils_buffered, drivers_gpd, how='left', op='intersects')


/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


In [12]:
reserve_data = matches_within_distance.groupby('unique_id').sample(n=1)


In [13]:
# Use the function with your DataFrame
reserve_data = matches_within_distance.groupby('unique_id').sample(n=1)


prediction_list = []

for i in range(10):
  matched_data = matches_within_distance.groupby('unique_id').sample(n=1)
  matched_data = matched_data.reset_index(drop=True)
  model, test_loss, r_squared, inversed_predictions, scaler = preprocess_and_train_model(matched_data)

  predictions = predict_on_reserve(reserve_data,model,scaler)
  prediction_list.append(predictions)


<ipython-input-4-352fa283bc47>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_cols.replace('', np.nan, inplace=True)


Epoch 1/100
5/5 [==============================] - 2s 73ms/step - loss: 2.1275 - val_loss: 0.0204
Epoch 2/100
5/5 [==============================] - 0s 17ms/step - loss: 2.2841 - val_loss: 0.0243
Epoch 3/100
5/5 [==============================] - 0s 17ms/step - loss: 2.0613 - val_loss: 0.0285
Epoch 4/100
5/5 [==============================] - 0s 18ms/step - loss: 1.6411 - val_loss: 0.0342
Epoch 5/100
5/5 [==============================] - 0s 17ms/step - loss: 1.7353 - val_loss: 0.0362
Epoch 6/100
5/5 [==============================] - 0s 18ms/step - loss: 2.2026 - val_loss: 0.0401
Epoch 7/100
5/5 [==============================] - 0s 17ms/step - loss: 1.6755 - val_loss: 0.0372
Epoch 8/100
5/5 [==============================] - 0s 17ms/step - loss: 1.4697 - val_loss: 0.0403
Epoch 9/100
5/5 [==============================] - 0s 18ms/step - loss: 1.3085 - val_loss: 0.0398
Epoch 10/100
5/5 [==============================] - 0s 17ms/step - loss: 1.1024 - val_loss: 0.0385
Epoch 11/100
5/5 [=

<ipython-input-5-8459f0b8cc1d>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_cols.replace('', np.nan, inplace=True)
<ipython-input-4-352fa283bc47>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_cols.replace('', np.nan, inplace=True)


Epoch 1/100
5/5 [==============================] - 2s 65ms/step - loss: 2.7283 - val_loss: 0.0413
Epoch 2/100
5/5 [==============================] - 0s 18ms/step - loss: 2.5083 - val_loss: 0.0343
Epoch 3/100
5/5 [==============================] - 0s 19ms/step - loss: 2.1619 - val_loss: 0.0314
Epoch 4/100
5/5 [==============================] - 0s 19ms/step - loss: 2.0212 - val_loss: 0.0365
Epoch 5/100
5/5 [==============================] - 0s 21ms/step - loss: 1.8004 - val_loss: 0.0349
Epoch 6/100
5/5 [==============================] - 0s 21ms/step - loss: 1.4079 - val_loss: 0.0328
Epoch 7/100
5/5 [==============================] - 0s 20ms/step - loss: 1.5525 - val_loss: 0.0340
Epoch 8/100
5/5 [==============================] - 0s 18ms/step - loss: 1.1318 - val_loss: 0.0318
Epoch 9/100
5/5 [==============================] - 0s 19ms/step - loss: 1.5049 - val_loss: 0.0327
Epoch 10/100
5/5 [==============================] - 0s 19ms/step - loss: 1.0873 - val_loss: 0.0321
Epoch 11/100
5/5 [=

<ipython-input-5-8459f0b8cc1d>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_cols.replace('', np.nan, inplace=True)
<ipython-input-4-352fa283bc47>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_cols.replace('', np.nan, inplace=True)


...

<ipython-input-5-8459f0b8cc1d>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_cols.replace('', np.nan, inplace=True)
<ipython-input-4-352fa283bc47>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_cols.replace('', np.nan, inplace=True)


Epoch 1/100
5/5 [==============================] - 2s 64ms/step - loss: 2.4693 - val_loss: 0.0567
Epoch 2/100
5/5 [==============================] - 0s 21ms/step - loss: 2.6759 - val_loss: 0.0216
Epoch 3/100
5/5 [==============================] - 0s 18ms/step - loss: 2.3667 - val_loss: 0.0316
Epoch 4/100
5/5 [==============================] - 0s 20ms/step - loss: 1.8557 - val_loss: 0.0397
Epoch 5/100
5/5 [==============================] - 0s 18ms/step - loss: 1.5211 - val_loss: 0.0358
Epoch 6/100
5/5 [==============================] - 0s 18ms/step - loss: 1.6018 - val_loss: 0.0350
Epoch 7/100
5/5 [==============================] - 0s 19ms/step - loss: 1.3593 - val_loss: 0.0387
Epoch 8/100
5/5 [==============================] - 0s 19ms/step - loss: 1.1822 - val_loss: 0.0351
Epoch 9/100
5/5 [==============================] - 0s 19ms/step - loss: 1.5003 - val_loss: 0.0373
Epoch 10/100
5/5 [==============================] - 0s 18ms/step - loss: 1.1100 - val_loss: 0.0366
Epoch 11/100
5/5 [=

<ipython-input-5-8459f0b8cc1d>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_cols.replace('', np.nan, inplace=True)
<ipython-input-4-352fa283bc47>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_cols.replace('', np.nan, inplace=True)


Epoch 1/100
5/5 [==============================] - 2s 71ms/step - loss: 3.2995 - val_loss: 0.0192
Epoch 2/100
5/5 [==============================] - 0s 22ms/step - loss: 2.2412 - val_loss: 0.0209
Epoch 3/100
5/5 [==============================] - 0s 23ms/step - loss: 2.1892 - val_loss: 0.0303
Epoch 4/100
5/5 [==============================] - 0s 20ms/step - loss: 1.7797 - val_loss: 0.0239
Epoch 5/100
5/5 [==============================] - 0s 20ms/step - loss: 2.6092 - val_loss: 0.0299
Epoch 6/100
5/5 [==============================] - 0s 23ms/step - loss: 1.6401 - val_loss: 0.0346
Epoch 7/100
5/5 [==============================] - 0s 22ms/step - loss: 1.7774 - val_loss: 0.0299
Epoch 8/100
5/5 [==============================] - 0s 23ms/step - loss: 1.8318 - val_loss: 0.0293
Epoch 9/100
5/5 [==============================] - 0s 24ms/step - loss: 1.3527 - val_loss: 0.0317
Epoch 10/100
5/5 [==============================] - 0s 24ms/step - loss: 1.3116 - val_loss: 0.0318
Epoch 11/100
5/5 [=

<ipython-input-5-8459f0b8cc1d>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_cols.replace('', np.nan, inplace=True)


In [14]:
combined_array = np.array([predictions['inversed_predictions'].values for predictions in prediction_list])

In [15]:
combined_array[combined_array < 0 ] = 0

In [16]:
arr_min = np.min(combined_array.flatten())
arr_max = np.max(combined_array.flatten())

In [17]:
dist_array

NameError: name 'dist_array' is not defined

In [ ]:
dist_array = [np.sort(np.array([i[j] for  i in combined_array])) for j in range(combined_array.shape[1])]
norm_dist_array =  [np.sort((np.array([i[j] for  i in combined_array]) - arr_min) / (arr_max-arr_min)) for j in range(combined_array.shape[1])]

In [ ]:
array = norm_dist_array[0]
# Generating y values as indices (percentiles in this context)
y_values = np.linspace(1, 100, len(array))

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(array, y_values, marker='o', linestyle='-', color='b')
plt.title('Index (Percentile) by Value')
plt.xlabel('Value')
plt.ylabel('Percentile')
plt.grid(True)
plt.show()

# Plotting
plt.figure(figsize=(10, 6))
plt.hist(array, bins=len(array), edgecolor='black')
plt.title('Histogram of Values')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()



In [ ]:
keep_cols = [21,22,23,24,25,26,27,28, 29, 31,32, 33,34,36,36,37,38,39,40,41,42,43,44,45,46,47, 48]

numeric_cols = pred_data.iloc[:,keep_cols]

scaler = MinMaxScaler()

scaler.fit(numeric_cols)

scaled_numeric_cols = scaler.transform(numeric_cols)

scaled_numeric_df = pd.DataFrame(scaled_numeric_cols, columns=numeric_cols.columns, index=numeric_cols.index)

# scaled_numeric_df['dist_array'] = dist_array

scaled_numeric_df['norm_dist_array'] = norm_dist_array

id_fields = pred_data[['source_dataset', 'island', 'soil_column_id', 'unique_id', 'depth_top', 'depth_bottom', 'depth_adj_bottom', 'latitude', 'longitude']]

numeric_df = pd.concat([id_fields, scaled_numeric_df], axis=1)


In [ ]:
numeric_df.columns

In [ ]:
# Model Definition
def build_model(input_shape,output_shape):
    model = Sequential([
        Dense(256, activation='relu', input_dim=input_shape),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        Dense(512, activation='relu'),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        Dense(1024, activation='relu'),
        BatchNormalization(),
        LeakyReLU(alpha=0.2),
        Dropout(0.3),
        Dense(output_shape, activation='linear')  # Adjust output layer according to your needs
    ])
    return model


In [ ]:
scaled_numeric_df['norm_dist_array'] = scaled_numeric_df['norm_dist_array'].to_list()

In [ ]:
scaled_numeric_df.dropna(inplace=True)

In [ ]:
norm_dist_array = np.array([i for i in scaled_numeric_df['norm_dist_array']])

In [ ]:

X = scaled_numeric_df.iloc[:, :-1]
y = scaled_numeric_df.iloc[:, -1]

x = np.array(X)
y = np.array(norm_dist_array)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# print(X_train.shape[1])
model = build_model(X_train.shape[1],output_shape=10)

In [ ]:
y_train.shape

In [ ]:


# model = build_model((X_train.shape[1]))
# model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error',metrics = ['mae'])
# model.fit(X_train, y_train, validation_split=0.2, epochs=100, batch_size=128, verbose=1)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='mean_squared_error',metrics = ['mae'])
model.fit(X_train, y_train, validation_split=0.2, epochs=600, batch_size=128, verbose=1)
model.compile(optimizer=Adam(learning_rate=0.00001), loss='mean_squared_error',metrics = ['mae'])
model.fit(X_train, y_train, validation_split=0.2, epochs=600, batch_size=128, verbose=1)

test_loss = model.evaluate(X_test, y_test)

# Predictions and Evaluation
predictions = model.predict(X_test)
# dummy_array = np.full((len(predictions), 28), fill_value=0.5)  # Example for reverse scaling
# dummy_array[:, -1] = predictions
# inversed_predictions = scaler.inverse_transform(dummy_array)[:, -1]
# dummy_array[:, -1] = y_test
# inversed_truth = scaler.inverse_transform(dummy_array)[:, -1]
# r_squared = r2_score(inversed_truth, inversed_predictions)

# return model, test_loss, r_squared, inversed_predictions, scaler

In [ ]:
i = 4
for i in range(5):
  print(predictions[i])
  print(y_test[i])